In [1]:
# place imports here
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score, confusion_matrix, classification_report, accuracy_score

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv('../dataset/AppleStoreWithoutNoise.csv')
dataset.describe()

,size_bytes,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver
count,6.268000e+03,6268.000000,6.268000e+03,6268.000000,6268.000000,6268.000000
mean,2.057430e+08,1.821977,1.480381e+04,528.607371,4.049697,3.735801
std,3.526341e+08,6.128238,8.098468e+04,4196.699812,0.726943,1.399040
min,5.898240e+05,0.000000,1.000000e+00,0.000000,1.000000,0.000000
25%,5.171942e+07,0.000000,7.800000e+01,6.000000,4.000000,3.500000
50%,1.021292e+08,0.000000,5.125000e+02,37.000000,4.500000,4.500000
75%,1.884774e+08,2.990000,3.963500e+03,176.000000,4.500000,4.500000
max,4.025970e+09,299.990000,2.974676e+06,177050.000000,5.000000,5.000000


In [3]:
dataset['size_100kilo_bytes'] = dataset['size_bytes']/102400
dataset['price_100'] = dataset['price'] / 100;

mms = MinMaxScaler()
dataset['price_mms'] = mms.fit_transform(dataset[['price']])
dataset['rating_count_tot_mms'] = mms.fit_transform(dataset[['rating_count_tot']])
dataset['rating_count_ver_mms'] = mms.fit_transform(dataset[['rating_count_ver']])

In [4]:
dataset.describe()

,size_bytes,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,size_100kilo_bytes,price_100,price_mms,rating_count_tot_mms,rating_count_ver_mms
count,6.268000e+03,6268.000000,6.268000e+03,6268.000000,6268.000000,6268.000000,6268.000000,6268.000000,6268.000000,6268.000000,6268.000000
mean,2.057430e+08,1.821977,1.480381e+04,528.607371,4.049697,3.735801,2009.209099,0.018220,0.006073,0.004976,0.002986
std,3.526341e+08,6.128238,8.098468e+04,4196.699812,0.726943,1.399040,3443.692860,0.061282,0.020428,0.027225,0.023703
min,5.898240e+05,0.000000,1.000000e+00,0.000000,1.000000,0.000000,5.760000,0.000000,0.000000,0.000000,0.000000
25%,5.171942e+07,0.000000,7.800000e+01,6.000000,4.000000,3.500000,505.072500,0.000000,0.000000,0.000026,0.000034
50%,1.021292e+08,0.000000,5.125000e+02,37.000000,4.500000,4.500000,997.355000,0.000000,0.000000,0.000172,0.000209
75%,1.884774e+08,2.990000,3.963500e+03,176.000000,4.500000,4.500000,1840.600000,0.029900,0.009967,0.001332,0.000994
max,4.025970e+09,299.990000,2.974676e+06,177050.000000,5.000000,5.000000,39316.110000,2.999900,1.000000,1.000000,1.000000


In [5]:
X_npy = dataset[['price_mms', 'rating_count_tot_mms', 'rating_count_ver_mms', 'size_100kilo_bytes']]
X_npy.describe()

,price_mms,rating_count_tot_mms,rating_count_ver_mms,size_100kilo_bytes
count,6268.000000,6268.000000,6268.000000,6268.000000
mean,0.006073,0.004976,0.002986,2009.209099
std,0.020428,0.027225,0.023703,3443.692860
min,0.000000,0.000000,0.000000,5.760000
25%,0.000000,0.000026,0.000034,505.072500
50%,0.000000,0.000172,0.000209,997.355000
75%,0.009967,0.001332,0.000994,1840.600000
max,1.000000,1.000000,1.000000,39316.110000


In [6]:
y_npy = dataset[['user_rating']]
y_npy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6268 entries, 0 to 6267
Data columns (total 1 columns):
user_rating    6268 non-null float64
dtypes: float64(1)
memory usage: 49.0 KB


In [7]:
pd.options.mode.chained_assignment = None
y_npy['happy'] = np.where(y_npy['user_rating'] >= 4.5, 1, 0)
y_npy['unhappy'] = np.where(y_npy['user_rating'] < 4.5, 1, 0)
y_npy.head()

,user_rating,happy,unhappy
0,4.0,0,1
1,4.0,0,1
2,3.5,0,1
3,4.0,0,1
4,4.5,1,0


In [8]:
X = X_npy.values
y = y_npy[['happy', 'unhappy']].values

In [9]:
y.shape

(6268, 2)

In [15]:
def model_builder_sigmoid():
    model = Sequential()
    model.add(Dense(2, input_shape=(4,), activation='sigmoid'))
    model.compile(SGD(lr=0.2), 
                  'binary_crossentropy', 
                  metrics=['accuracy'])
    return model

In [16]:
model = KerasClassifier(build_fn=model_builder_sigmoid, epochs=50, verbose=0)

In [17]:
cv = KFold(3, shuffle=True)
scores = cross_val_score(model, X, y, cv=cv)
print("Cross validation accuracy is {:0.4f} +- {:0.4f}".format(scores.mean(), scores.std()))

Cross validation accuracy is 0.4979 +- 0.0029


In [12]:
X.shape

(6268, 4)

In [13]:
y.shape

(6268, 1)

In [14]:
model = Sequential()
model.add(Dense(1, input_dim=4, activation='sigmoid'))
model.compile(Adam(lr=0.5), 'binary_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
5014/5014 [==============================] - 0s - loss: 8.1362 - acc: 0.4946     
Epoch 2/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 3/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 4/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 5/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 6/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 7/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 8/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 9/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 10/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 11/100
5014/5014 [==============================] - 0

5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 88/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 89/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 90/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 91/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 92/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 93/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 94/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 95/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 96/100
5014/5014 [==============================] - 0s - loss: 8.1523 - acc: 0.4942     
Epoch 97/100
5014/5014 [==============================] - 0s - 

In [58]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("The Accuracy score on the Train set is:\t{:0.3f}".format(accuracy_score(y_train, y_train_pred.round())))
print("The Accuracy score on the Test set is:\t{:0.3f}".format(accuracy_score(y_test, y_test_pred.round())))

The Accuracy score on the Train set is:	0.925
The Accuracy score on the Test set is:	0.906
